In [3]:
import glob
import numpy as np
import pandas as pd
import scipy.signal as sig

In [102]:
def search_sessions_by_media(mediafile):
    files = map(lambda x: np.load(x, encoding="latin1"), glob.glob("../export/data/gaze_data_*.npz"))
    return [ f for f in files if f['session_info'].tolist()["mediaFile"] == '{}.avi'.format(mediafile) ]

def get_feature_data_by_mediafile(mediafile):
    valence = pd.DataFrame.from_csv('valence_combined_feature_{}.csv'.format(mediafile))
    arousal = pd.DataFrame.from_csv('arousal_combined_feature_{}.csv'.format(mediafile))
    return valence, arousal

def export_gaze_data(mediafile):

    valid_files = search_sessions_by_media(mediafile)
    
    valence, arousal = get_feature_data_by_mediafile(mediafile)
    
    max_len = valence.shape[0]
    
    coordinates_data = None
    fixations_data = None

    for f in valid_files:

        sid = f["session_info"].tolist().get("sessionId")
            
        coordinates = pd.DataFrame(f["coordinates"], columns=["x", "y"])[:max_len]
        fixations = pd.DataFrame(f["fixations"], columns=["x", "y", 'd'])[:max_len]
        distance = pd.DataFrame(f["mean_distance"][0], columns=['distance'])[:max_len]

        coordinates[sid] = list(zip(coordinates['x'], coordinates['y'], distance['distance'], valence[sid]))
        fixations[sid] = list(zip(fixations['x'], fixations['y'], fixations['d'], distance['distance'], valence[sid]))

        coordinates = coordinates.drop(["x", "y"], axis=1)
        fixations = fixations.drop(["x", "y", 'd'], axis=1)

        if coordinates_data is None:
            coordinates_data = coordinates
            fixations_data = fixations
        else:
            coordinates_data = pd.concat([coordinates_data, coordinates], axis=1)
            fixations_data = pd.concat([fixations_data, fixations], axis=1)
            
    return coordinates_data, fixations_data

In [111]:
for mediafile in [30, 53, 69, 90, 111]:
    coord_data, fix_data = export_gaze_data(mediafile=mediafile)
    coord_data.to_csv("sm_data_coord_{}.csv".format(mediafile))
    fix_data.to_csv("sm_data_fix_{}.csv".format(mediafile))

/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  import sys
